<h2>Classification of structured data with feature columns</h2>
<p>In this topic, we will understand how to classify structured data (example, CSV). For this we are using keras to create the model and feature columns as a bridge to map from columns in a csv to features. These features used to train the model. The steps are:</p>
<ul><li>Load a csv file using Pandas</li>
<li>Build an input pipeline to batch and shuffle the rows using tf.data</li>
<li>Map from columns in the CSV to features for training.</li>
<li>Build, train and evaluate the model using Keras</li></ul>


<h3>Load a csv file using Pandas</h3>
<h4>Dataset </h4>
<p>We will be using simplified version of the PetFinder dataset from googleapis website. There are many number of rows in the dataset which describe a pet and columns describe an attribute. We are using this dataset to predict the the speed at which the pet will be adopted. In the dataset, there are both numerical and categorical columns.</p> 

In [1]:
#Importing required libraries
import numpy as np
import pandas as pd

import tensorflow as tf
import pathlib

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

<h4>Use Pandas to create a dataframe</h4>
<p>We will be using Python library pandas for downloading the dataset using the link and load it into a dataframe. Pandas helpful for loading and working with structured data.</p>

In [2]:
# Specifying the url where the dataset available
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'

#Specifying the file name which we will use
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

#Get the zip file from url and extract it 
tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
#Loading the csv file to a dataframe
dataframe = pd.read_csv(csv_file)

1671168/1668792 [==============================] - 0s 0us/step


In [3]:
#Displaying the first 5 rows of dataframe
dataframe.head()

Type  Age  ... PhotoAmt AdoptionSpeed
0  Cat    3  ...        1             2
1  Cat    1  ...        2             0
2  Dog    1  ...        7             3
3  Dog    4  ...        8             2
4  Dog    1  ...        3             2

[5 rows x 15 columns]

<h4>Create target variable</h4>
<p> Our goal is to predict the speed at which a pet will be adopted. (For example, One week, One month, Three week etc). So we will transform this into a binary classification problem, Whether the pet is adaptable or not. Then O will indicate the pet is not adoptable and 1 indicates it is adoptable.</p>

In [4]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

<h4>Split the dataset</h4>
<p>We have only one csv file so, it will be split the data into train, test and validation sets.</p>

In [5]:
#Dividing the set into train and test
train, test = train_test_split(dataframe, test_size=0.2)

#Dividing the set into train and validation
train, val = train_test_split(train, test_size=0.2)

#Displaying the length of train, validaton and test sets
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7383 train examples
1846 validation examples
2308 test examples


<h3>Creating a input pipeline using tf.data</h3>
<p> Here we wrap the dataframes with tf.data. This will enable the feature column as a bridge map from the column in the dataframe to features used to train the model. If the csv file is very large, we have to use tf.data to reaf it from the disk.</p>

In [6]:
# A  method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()

  #Delete the target label from the dataframe
  labels = dataframe.pop('target')

  #Adding tf.data wrap to the dataframe
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [7]:
# A small batch sized is used for demonstration purposes
batch_size = 5 

#Assigning train, validation and test dataset with batch_size is 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

<h4>Understand the input pipeline</h4>
<p>In the previous step we have created the input pipeline for the dataframe. We used batchsize of 5 units</p>

In [8]:
#Display the format of data after tf.data wrapped
for feature_batch, label_batch in train_ds.take(1):
  #Displaying the dictionary of column names
  print('Every feature:', list(feature_batch.keys()))

  #Printing the feature batch age value
  print('A batch of ages:', feature_batch['Age'])

  #Printing the label batch value or target values
  print('A batch of targets:', label_batch )

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
A batch of ages: tf.Tensor([ 3  3 12 12  2], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 1 0 1 1], shape=(5,), dtype=int64)


<h3>Demonstrate different type of feature columns</h3>
<p> In TensorFlow library, there are many type of feature columns, We will be creating different type of feature columns and demonstrate how they transform column from the dataframe</p>

In [9]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [10]:
# A method to create a feature column and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

<h4>Numeric columns</h4>
<p> Numerical columns is the simplest type of column. The output of feature column becomes input to the model. It is used to represent real valued features. In using numerical columns, the model will receive the column value from the dataframe withou any change. We wil be using the demo function defined above and we see how each column from the dataframe is transformed.</p> 

In [11]:
# Applying numeric_column function on PhotoAmt column
photo_count = feature_column.numeric_column('PhotoAmt')
demo(photo_count)

[[3.]
 [0.]
 [4.]
 [5.]
 [2.]]


<h4>Bucketized columns</h4>
<p>We can feed the number to the model in directly or by specifying the ranges. The value is splitted into different categories based on numeric ranges. For example, the person's age data can be split into several buckets using bucketized column. </p>

In [12]:
#Applying numerical_column function to Age column 
age = feature_column.numeric_column('Age')

#Applying bucketized column function to the age variable with boudaries 1, 3 and 5
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 3, 5])

#Applying demo user function for the age_buckets
demo(age_buckets)

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


<h4>Categorical columns</h4>
<p>In the dataset, the strings are not feeded directly into the model. We first map them to the numerical values. The categorical vocabulary columns provide a way to represent strings as a one-hot vector (similar like buckets). The vocabulary can be passed as a list using <i>categorical_column_with_vocabulary_list</i>, or loaded from a file using <i>categorical_column_with_vocabular_file</i>. </p>

In [13]:
#Applying categorical_column_with_vocabulary_list by passing the list
animal_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', ['Cat', 'Dog'])

#Applying indicator_column function to represent animal_type as one-hot vector
animal_type_one_hot = feature_column.indicator_column(animal_type)

#Applying the demo function to the one-hot encoded animal type
demo(animal_type_one_hot)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


<h4>Embedding columns</h4>
<p>Sometimes, there are more values per category. For many reasons, the number of categories grow large, it becomes infeasibleto train a neural network using one-hot encodings. Instead of representing the data as one-hot vector of many dimensions, we can use embedding columns. It represents the data as a lower-dimensional, dense vector in which each cell contain any number (fro 0 to 9).-<h4<<jdkfelfklef12o1[hsjjsmcnvknvmcv <<<<<<

In [14]:
# Notice the input to the embedding column is the categorical column
# we previously created
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'Breed1', dataframe.Breed1.unique())

# Applying embedding_column function to breed1 with 8 dimensions
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
demo(breed1_embedding)

[[-0.41942    -0.4473536  -0.42639008 -0.09102362  0.10016519  0.55932945
   0.43054193 -0.04128454]
 [-0.41942    -0.4473536  -0.42639008 -0.09102362  0.10016519  0.55932945
   0.43054193 -0.04128454]
 [-0.41942    -0.4473536  -0.42639008 -0.09102362  0.10016519  0.55932945
   0.43054193 -0.04128454]
 [-0.41942    -0.4473536  -0.42639008 -0.09102362  0.10016519  0.55932945
   0.43054193 -0.04128454]
 [-0.18831994 -0.12173289 -0.07174521  0.1671272   0.11718646 -0.31742236
  -0.13754228 -0.06349101]]


<h4>Hashed feature volumns</h4>
<p>This type of feature column is another way to represent a categorical column with a large number of values. <i>categorical_column_with_hash_bucket</i> calculates a hash value of the input, then selects one of the hash_bucket_size buckets to encode a string. We do not need to provide the vocabulary while using this feature column and we can make the number of hash_buckets significantly smaller than the number of actual categories to save space.</p>

In [15]:
#Applying hashed feature columns with bucket size 10 for Breed1 column
breed1_hashed = feature_column.categorical_column_with_hash_bucket(
      'Breed1', hash_bucket_size=10)

#Applying demo function to breed1_hashed
demo(feature_column.indicator_column(breed1_hashed))

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


<h4>Cross feature columns</h4>
<p>This feature column combines number of features into a single feature. It will enables a model to learn separate weights for each combination of features. In this example, we create a new feature that is the cross of AGe and Type column of the data. The crossed_column does not build the full table of all possible combination (which become very large). Instead, it is backed by a hashed_column, so we can choose how large the table is:</p>

In [16]:
# Applying cross feature columns to Age and type columns with bucket size 10
crossed_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=10)

#Apply demo function to cross_feature
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


<h3>Choose which column to use</h3>
<p> We have seen how to use several types of feature columns. Now we will use them to train model. This topic show complete code needed to work with feature columns. We have selected a few columns to train our model below arbitrarily.</p> 

In [17]:
#Declaring the list to add diferent feature columns
feature_columns = []

# numeric column for the columns PhotoAmt, Fee and Age 
for header in ['PhotoAmt', 'Fee', 'Age']:
  feature_columns.append(feature_column.numeric_column(header))

In [18]:
# bucketized column for Age
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 2, 3, 4, 5])
feature_columns.append(age_buckets)

In [19]:
# Categorical columns for the below list of columns
indicator_column_names = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                          'FurLength', 'Vaccinated', 'Sterilized', 'Health']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [20]:
# embedding columns for Breed 1 column with 8 dimensions
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
feature_columns.append(breed1_embedding)

In [21]:
# crossed columns for Age and Type column with hash_bucket_size 100
age_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(age_type_feature))

<h4>Creating a feature layer</h4>
<p>After defining the feature columns, we will create a feature layer. We will use a <b>DenseFeatures</b> layer to input feature columns to the model.</p>

In [22]:
# feature_columns list is passed as input to the DenseFeatures function
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [23]:
#Creating the input pipeline with batch size 32
batch_size = 32

train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

<h3>Create, Compile and Train the model</h3>

In [24]:
# Creating the model with feature layer, 3 Dense layer and 1 Dropout layer
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

# Compiling the model with Adam optimizer
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Training the model with 10 epochs using fit() function
model.fit(train_ds,
          validation_data=val_ds,
          epochs=50)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
227/231 [============================>.] - ETA: 0s - loss: 0.7211 - accuracy: 0.6632WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Type': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'Age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'Breed1': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=string>, 'Gender': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'Color1': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=string>, 'Color2': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'MaturitySize': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'FurLength': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=string>, 'Vaccinated': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'Sterilized': <tf.Tensor 'ExpandDims_10

In [25]:
# Evaluating the model with test dataset
loss, accuracy = model.evaluate(test_ds)

#Displaying the accuracy of the model
print("Accuracy", accuracy)

73/73 [==============================] - 0s 2ms/step - loss: 0.5333 - accuracy: 0.7006
Accuracy 0.7006065845489502
